# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(".\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,4.02,81,100,8.75,IS,1717851044
1,1,bilibino,68.0546,166.4372,3.67,57,3,1.28,RU,1717851045
2,2,mount pearl,47.5166,-52.7813,12.20,96,100,4.63,CA,1717850960
3,3,hermanus,-34.4187,19.2345,18.22,63,100,2.40,ZA,1717851045
4,4,cabedelo,-6.9811,-34.8339,27.37,80,63,5.55,BR,1717851045


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"*100,
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df.dropna(inplace=True)
       
# Display sample data
city_data_df

C:\Users\rmt20\AppData\Local\Temp\ipykernel_56680\3650330861.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,hawaiian paradise park,19.5933,-154.9731,21.73,92,0,2.57,US,1717851048
88,88,haiku-pauwela,20.9219,-156.3051,23.36,84,0,0.45,US,1717851052
104,104,maun,-19.9833,23.4167,24.98,16,0,3.60,BW,1717851053
122,122,stephenville,32.2207,-98.2023,25.64,84,0,2.06,US,1717851054
177,177,fort bragg,35.1390,-79.0060,24.00,59,0,2.57,US,1717851062
231,231,westport,41.1415,-73.3579,21.87,64,0,4.12,US,1717851068
294,294,santa luzia,-19.7697,-43.8514,22.10,71,0,4.12,BR,1717851075
381,381,ojinaga,29.5667,-104.4167,26.43,53,0,2.06,MX,1717851085
410,410,nkove,-7.9500,30.8500,26.93,42,0,1.91,TZ,1717851088
460,460,ibia,-19.4783,-46.5389,21.06,44,0,2.79,BR,1717851095


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,isafjordur,IS,66.0755,-23.1240,81,
1,bilibino,RU,68.0546,166.4372,57,
2,mount pearl,CA,47.5166,-52.7813,96,
3,hermanus,ZA,-34.4187,19.2345,63,
4,cabedelo,BR,-6.9811,-34.8339,80,
...,...,...,...,...,...,...
565,duong dong,VN,10.2167,103.9667,74,
566,neryungri,RU,56.6709,124.6491,84,
567,nev'yansk,RU,57.4953,60.2112,63,
568,klamath falls,US,42.2249,-121.7817,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
isafjordur - nearest hotel: Hótel Horn
bilibino - nearest hotel: No hotel found
mount pearl - nearest hotel: Sandman Signature St. John's Hotel
hermanus - nearest hotel: Aloe guest house
cabedelo - nearest hotel: No hotel found
opuwo - nearest hotel: Ohakane Lodge
timimoun - nearest hotel: أناف
posse - nearest hotel: Palmeiras Hotel
coahuayana de hidalgo - nearest hotel: No hotel found
hinton - nearest hotel: Timberland Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
remire-montjoly - nearest hotel: Complexe Belova
waitangi - nearest hotel: Hotel Chathams
taber - nearest hotel: No hotel found
mejit - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
kenai - nearest hotel: Beluga Lookout Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
kabansk - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
novaya igirma - nearest hotel: Тайга
capo d'orlando - nearest hotel: Hotel Sant'Andrea
p

,City,Country,Lat,Lng,Humidity,Hotel Name
0,isafjordur,IS,66.0755,-23.1240,81,Hótel Horn
1,bilibino,RU,68.0546,166.4372,57,No hotel found
2,mount pearl,CA,47.5166,-52.7813,96,Sandman Signature St. John's Hotel
3,hermanus,ZA,-34.4187,19.2345,63,Aloe guest house
4,cabedelo,BR,-6.9811,-34.8339,80,No hotel found
...,...,...,...,...,...,...
565,duong dong,VN,10.2167,103.9667,74,Kim Hông Ngoc
566,neryungri,RU,56.6709,124.6491,84,DeLuxe
567,nev'yansk,RU,57.4953,60.2112,63,Старый соболь
568,klamath falls,US,42.2249,-121.7817,83,Quality Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"*100,
    scale = 0.01,
    color = "City",
    hover_cols=["Lng", "Lat", "City", "Country","Hotel Name"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)